<a href="https://colab.research.google.com/github/anagomesq/ana-clara/blob/main/atividadebancodedados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Criando conexão
connection = sqlite3.connect("mapeamento_cultural.sqlite")

query = """
CREATE TABLE endereco (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rua VARCHAR(100),
    numero VARCHAR(10),
    CEP VARCHAR(10),
    regiao VARCHAR(50)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE pontos_historicos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    descricao TEXT,
    endereco_id INT,
    FOREIGN KEY (endereco_id) REFERENCES endereco(id)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE grupo_cultural (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome VARCHAR(100),
    descricao TEXT,
    fotos TEXT,
    contato VARCHAR(100)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE evento (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATE,
    horario TIME,
    local VARCHAR(100),
    status VARCHAR(50),
    url_ingresso VARCHAR(255)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE localiza (
    grupo_id INTEGER,
    ponto_id INTEGER,
    PRIMARY KEY (grupo_id, ponto_id),
    FOREIGN KEY (grupo_id) REFERENCES grupo_cultural(id),
    FOREIGN KEY (ponto_id) REFERENCES pontos_historicos(id)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE apresenta (
    grupo_id INTEGER,
    evento_id INTEGER,
    grupo_responsavel VARCHAR(100),
    PRIMARY KEY (grupo_id, evento_id),
    FOREIGN KEY (grupo_id) REFERENCES grupo_cultural(id),
    FOREIGN KEY (evento_id) REFERENCES evento(id)
);
"""
execute_query(connection, query)

Query executada.
Query executada.
Query executada.
Query executada.
Query executada.
Query executada.


**-- EDITE ESSA CÉLULA --**

**CONSULTA 1**

* Objetivo da consulta:contar o número de grupos culturais que atuam em cada ponto histórico
* Descrição da ação esperada ao rodar a consulta:mostra quais pontos históricos têm maior atividade cultural.


In [ ]:
select
    p.id AS ponto_id,
    p.descricao,
    count(l.grupo_id) AS total_grupos
from
    pontos_historicos p
left join
    localiza l ON p.id = l.ponto_id
group by
    p.id, p.descricao;

CONSULTA 2

* Objetivo da consulta :listar cada grupo cultural com o número total de eventos em que ele se apresentoy
* Descrição da ação esperada ao rodar a consulta: mostra os eventos mais engajados

In [ ]:
select
    g.nome AS grupo_cultural,
    e.local AS local_evento,
    p.descricao AS ponto_historico
from
    grupo_cultural g
join
    apresenta a ON g.id = a.grupo_id
join
    evento e ON a.evento_id = e.id
join
    localiza l ON g.id = l.grupo_id
JSONFileConfigLoader
    pontos_historicos p ON l.ponto_id = p.id;

In [ ]:
select
    g.id AS grupo_id,
    g.nome,
    count(a.evento_id) as total_eventos
from
    grupo_cultural g
left join
    apresenta a ON g.id = a.grupo_id
group by
    g.id, g.nome;

CONSULTA 3
* A view view_eventos_culturais mostra todos os eventos culturais agendados, junto com os grupos culturais responsáveis, informações de contato e detalhes do evento. É importante para vizualizar rapidamente os eventos. Torna o acesso mais organizado e eficiente.

In [ ]:
create view view_eventos_culturais as
select
    g.nome as grupo,
    g.contato,
    e.data,
    e.horario,
    e.local,
    e.status,
    e.url_ingresso
from grupo_cultural g
join apresenta a ON g.id = a.grupo_id
join evento e ON e.id = a.evento_id;


CONSULTA 4
* No meu projeto PI3, a criação e atualização de grupos culturais é uma ação frequente, que envolve mostrar  se o grupo já existe antes de inserir para evitar mais deu um grupo. Usar uma procedure automatiza esse processo, sem repetir.

Ao executar essa procedure, ela recebe os dados do grupo cultural, verifica se já existe um grupo com o mesmo nome e, caso não exista, insere o grupo na tabela. Se existir, ela retorna uma mensagem avisando que o grupo já está cadastrado.

In [ ]:
create procedure inserir_grupo_cultural (
    in nome_grupo varchar(100),
    in descricao text,
    in contato varchar(100),
    OUT mensagem varchar(255)
)
begin
    if exists (select 1 from grupo_cultural where nome = nome_grupo) then
        set mensagem = 'grupo cultural já cadastrado.';
    else
        insert into grupo_cultural (nome, descricao, contato)
        velues (nome_grupo, descricao, contato);
        set mensagem = 'grupo cultural inserido arrasou ;)';
    end if;
end;


CONSULTA 5
  * No PI3, a gente precisa saber se o evento está programado para o futuro, para diferenciar os eventos antigos e os novos

  A function recebe a data de um evento e retorna TRUE se o evento ainda não ocorreu (data futura), e FALSE caso contrário.

In [ ]:
delimiter $$

create function evento_ativo(p_data_evento date)
returns tinyint(1)
deterministic
begin
    return p_data_evento >= curdate();
end$$

delimiter ;


CONSULTA 6
* Para manter a integridade do banco e garantir que todo evento novo comece com o status correto, o trigger automatiza a definição do status inicial sempre que um evento é inserido. Isso é para evitar erros.


O trigger é acionado após a inserção de um novo registro na tabela evento e automaticamente atualiza o campo status do evento para “Pendente”.

In [ ]:
delimiter $$

create trigger definir_status_evento
before insert on evento
for each row
begin
    set new.status = 'pendente';
end$$

delimiter ;